# Initialisation

In [2]:
# # conda environment setup - USE ISAACGYM env
# !conda create -n vid2player python=3.7
# !source activate vid2player 

###  The Colab notebook might automatically restart. You can directly run the next block. ###

In [3]:
# prompt: Check cuda version, display cuda device (gpu)
!nvcc --version
!nvidia-smi


/bin/bash: line 1: nvcc: command not found
Wed Jun 19 15:41:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080        Off | 00000000:01:00.0 Off |                  N/A |
| 18%   30C    P8              17W / 215W |     10MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+------------------------

# IsaacGym Install

In [4]:
# """
# Install IsaacGym Python Package (local package on GDrive)
# """
# from google.colab import drive
# drive.mount('/content/drive')
# try:
#   import isaacgym
# except:
#   !source activate vid2player && pip install -q -e /content/drive/MyDrive/Colab/isaacgym/isaacgym/python

In [5]:
# install gcc (and g++) version 9, and make 'gcc' use that version. Then, check the current version of gcc to check

# !apt-get install gcc-9 g++-9
# !update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 900
# !update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 900
# !update-alternatives --config gcc
# !gcc --version
# !apt install -y python3.7-dev
# !apt autoremove

# vid2player3d Install

In [6]:
# clone vid2player3d
import os
if not os.path.exists('./vid2player3d'):
  !git clone https://github.com/nv-tlabs/vid2player3d.git 

# create a symlink cause its easier
# !ln -s /content/drive/MyDrive/Colab/vid2player3d ./vid2player3d


In [7]:
# install python dependencies
# !source activate vid2player && bash ./vid2player3d/install.sh
!bash ./vid2player3d/install.sh

ERROR: Could not find a version that satisfies the requirement vtk==9.1.0 (from versions: none)
ERROR: No matching distribution found for vtk==9.1.0


         See 'conda install --help' for details about the --force-reinstall
         and --clobber flags.


Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /nvidia/noarch/repodata.json

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /pytorch/noarch/repodata.json

Channels:
 - conda-forge
Platform: linux-64

/ Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /conda-forge/noarch/repodata.json.zst

- Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken 

In [8]:
import os
if not os.path.exists("./mujoco210-linux-x86_64.tar.gz"):
  !wget https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz
!mkdir -p /home/charlie/.mujoco
!tar -xf mujoco210-linux-x86_64.tar.gz -C /home/charlie/.mujoco


# Steps For Tennis Sim:

1. Video Annotation

  a) All Third Party tools

2. Train low-level policy
  

In [9]:

import os
import subprocess

# Set the LD_LIBRARY_PATH environment variable
os.environ["LD_LIBRARY_PATH"] = "/home/charlie/.mujoco/mujoco210/bin" + ":" + os.environ.get("LD_LIBRARY_PATH", "")

In [ ]:
# Additional python lib
!pip install lxml joblib numpy-stl

# Install mujoco for visualization
!pip install -U  'mujoco-py<2.2,>=2.1'

# run in shell: sudo apt install patchelf

!pip install smplx

In [ ]:
# prompt: unzip from vid2player3d/data/amass/ACAAD.tar.bz2 to vid2player3d/data/amass/ACAAD
# !tar -xjf ./vid2player3d/vid2player/data/amass/ACCAD.tar.bz2 -C vid2player3d/vid2player/data/amass/


In [74]:
%cd vid2player3d
import os
import joblib
import numpy as np
from scipy.spatial.transform import Rotation as R

root_orient = 0

def process_files(amass_dir):
  """
  Walks through the amass directory and processes npz files.
  """
  result = {}
  for root, _, files in os.walk(amass_dir):
    filenames = [os.path.join(root, f) for f in files]
    for filename in filenames:
      if filename.endswith('.npz'):
        filepath = filename
        with np.load(filepath) as data:
          if all(key in data.keys() for key in ['trans', 'betas', 'gender', 'root_orient', 'pose_body']):
            global root_orient
            root_orient = data['root_orient']
            print(data['pose_body'].shape)
            pose_aa = np.concatenate([data['root_orient'], data['pose_body']], axis=-1)
            print(pose_aa.shape)
            return
            result[filename] = {
              'trans': data['trans'],
              'beta': data['betas'],
              'gender': data['gender'],
              'pose_aa': pose_aa
            }
      if filename.endswith('.pk'):
        filepath = filename
        with open(filepath, 'rb') as f:
          data = joblib.load(f)
          if all(key in data.keys() for key in ['transl', 'pred_betas', 'pred_thetas']):
            print(data.keys())
            poses = np.array([p.reshape(-1, 3, 3) for p in data['pred_thetas']])
            poses_rotvec = []
            for pose in poses:
              pose_rotvec = np.array([R.from_matrix(p).as_rotvec() for p in pose]).reshape(-1)
              poses_rotvec.append(pose_rotvec)
            poses_rotvec = np.array(poses_rotvec)

            mask = poses_rotvec.copy()
            mask[:, 3:] = 0
            mask[:, :3] = [0, np.radians(180), 0]
            poses_rotvec = poses_rotvec + mask

            betas = np.mean(data['pred_betas'], axis=0)
            result[filename] = {
              'trans': data['transl'],
              'beta': betas,
              'gender': 'male',
              'pose_aa': poses_rotvec
            }
  # Save the result as a pkl file
  joblib.dump(result, os.path.join(amass_dir, '..', 'processed_data.pkl'))
  print("Saved", len(result), "motions")

# Call the function with the amass directory path
process_files('./data/amass/ACCAD')
process_files('./data/tennis/sinner')




[Errno 2] No such file or directory: 'vid2player3d'
/home/charlie/Documents/ATPIL/Training/vid2player3d
(360, 63)
(360, 66)
(148, 72)
[[-3.05165478  3.11115279  0.03096239 ... -0.13668728 -0.05462345
   0.15152425]
 [-3.04005928  3.09963339  0.01545797 ... -0.13821409 -0.06422285
   0.11499881]
 [-3.07186226  3.07467839  0.09102398 ... -0.13170603 -0.02257139
   0.10587701]
 ...
 [-3.0421012   3.0784481   0.2440914  ... -0.13173811 -0.01906613
  -0.04170593]
 [-3.02789083  3.1111305   0.25658103 ... -0.13623233 -0.11097222
   0.01017017]
 [-2.99238213  3.13313134  0.24946242 ... -0.13843536 -0.06963674
  -0.01716262]]
Pred Beta STD: [0.08673272 0.11069424 0.10427027 0.1492436  0.09903067 0.06592883
 0.06319949 0.03521836 0.09282427 0.05347111]
(318, 72)
[[ 0.28808022  3.34872615 -2.98564004 ... -0.11708599  0.16261074
   0.19693258]
 [ 0.24562355  3.32184609 -2.987264   ... -0.11679365  0.15049285
   0.12618882]
 [ 0.24797099  3.33455083 -2.97587636 ... -0.11508572  0.1784753
   0.1734

In [75]:
# Convert into a format we can deal with
# Also had to change a bunch of poselib to poselib.poselib imports, change back after this apparently
!pwd
# !python uhc/utils/convert_amass_isaac.py --amass_data ./data/amass/processed_data.pkl --out_dir ./data/motion_lib/amass
!python uhc/utils/convert_amass_isaac.py --amass_data ./data/tennis/processed_data.pkl --num_motion_libs 1 --out_dir ./data/motion_lib/tennis

/home/charlie/Documents/ATPIL/Training/vid2player3d
Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
2024-06-21 11:36:14,081 - INFO - logger - logger initialized
MOVING MOTION DATA TO GPU, USING CACHE: True
AMASS data has 2 unique body shapes!
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.16it/s]


In [76]:
%cd /home/charlie/Documents/ATPIL/Training/vid2player3d
!python embodied_pose/run.py --cfg tennis_im --rl_device cuda:0 --test --checkpoint latest --test_motion_file data/motion_lib/tennis/mlib_part_00000.pth

/home/charlie/Documents/ATPIL/Training/vid2player3d


Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.1
Device count 1
/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /home/charlie/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /home/charlie/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...
Setting seed: 7
Started to play
2024-06-21 11:36:21,819 - INFO - logger - logger initialized
MOVING MOTION DATA TO GPU, USING CACHE: True
Loading motion file to cuda:0: data/motion_lib/tennis/mlib_part_00000.pth
Not connected to PVD
+++ Using GPU PhysX
Physics E

In [18]:
# !source activate vid2player && which python
import os
os.environ["LD_LIBRARY_PATH"] = "/home/charlie/miniforge3/envs/rlgpu/lib" + ":" + "/home/charlie/.mujoco/mujoco210/bin" + ":" + "/usr/lib/nvidia"
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libGLEW.so"

In [35]:
# Training Low-Level Imitation Policy
## Stage 1 : AMASS Motions
# 
%cd vid2player3d

!python embodied_pose/run.py --cfg amass_im --rl_device cuda:0 --test --headless

[Errno 2] No such file or directory: 'vid2player3d'
/home/charlie/Documents/ATPIL/Training/vid2player3d
Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.1
Device count 1
/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /home/charlie/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /home/charlie/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...
Setting seed: 7
Started to play
2024-06-20 14:39:21,669 - INFO - logger - logger initialized
MOVING MOTION DATA TO GPU, USING CACHE: True
Loading motion files

In [31]:
# Stage 2  : Tennis Motions
!python embodied_pose/run.py --cfg tennis_im --rl_device cuda:0 --headless

Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.1
Device count 1
/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /home/charlie/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /home/charlie/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...
Setting seed: 7
Started to train
2024-06-21 09:16:14,547 - INFO - logger - logger initialized
MOVING MOTION DATA TO GPU, USING CACHE: True
Loading motion files to cuda:0:
data/motion_lib/tennis/mlib_part_00000.pth
Not connected to PVD
+++ Using GPU PhysX
Physics

In [63]:
# Test
%cd /home/charlie/Documents/ATPIL/Training/vid2player3d
!python embodied_pose/run.py --cfg tennis_im --rl_device cuda:0 --test --checkpoint latest --test_motion_file data/motion_lib/tennis/mlib_part_00000.pth

/home/charlie/Documents/ATPIL/Training/vid2player3d
Importing module 'gym_38' (/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.1
Device count 1
/home/charlie/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /home/charlie/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /home/charlie/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...
Setting seed: 7
Started to play
2024-06-21 11:33:19,000 - INFO - logger - logger initialized
MOVING MOTION DATA TO GPU, USING CACHE: True
Loading motion file to cuda:0: data/motion_lib/tennis/mlib_part_00000.pt

# Train Motion Embedding

In [ ]:
!python vid2player/motion_vae/train.py

Traceback (most recent call last):
  File "vid2player/motion_vae/train.py", line 1, in <module>
    from motion_vae.config import *
ModuleNotFoundError: No module named 'motion_vae'


/home/charlie/Documents/ATPIL/Training/vid2player3d
